In [1]:
# Import essential libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
import networkx as nx
import matplotlib.pyplot as plt
import dgl
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.manifold import TSNE
import os # Thêm thư viện OS để tạo thư mục
from utils.attack_algo_utils import *
from utils.graph_utils import *
from utils.utils import *
# Đảm bảo tệp graph_utils.py nằm trong cùng thư mục

print("Đã import các thư viện.")

Using device: cuda
Đã import các thư viện.


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
import networkx as nx
import dgl
import torch

# --- 1. Tạo dữ liệu cho hai đồ thị mạng khác nhau ---
# (Đây là code GỐC từ yêu cầu của bạn)

# Đồ thị 1 (Lấy từ Graph.ipynb của bạn)
G_1 = nx.DiGraph()
G_1.add_nodes_from(["Data Server", "Pad", "Web Server", "Host 1", "Host 2", "Host 3", "File Server", "Router"])
node_attributes = {
    "Data Server": {"state": 0, "priority": 2}, "Pad": {"state": 0, "priority": 1},
    "Web Server": {"state": 0, "priority": 1}, "Host 1": {"state": 0, "priority": 1},
    "Host 2": {"state": 0, "priority": 0}, "Host 3": {"state": 0, "priority": 0},
    "File Server": {"state": 0, "priority": 0}, "Router": {"state": 0, "priority": 2}
}
nx.set_node_attributes(G_1, node_attributes)
edges = [
    ("Pad", "Host 1", {"user": 0.6, "root": 0.48}), ("Pad", "Host 2", {"user": 0.32, "root": 0.32}),
    ("Pad", "Host 3", {"user": 0.32, "root": 0.32}), ("Pad", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Host 1", "Pad", {"user": 0.6, "root": 0.6}), ("Host 1", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Host 1", "Host 2", {"user": 0.32, "root": 0.32}), ("Host 1", "Host 3", {"user": 0.32, "root": 0.32}),
    ("Host 2", "Host 3", {"user": 0.8, "root": 0.8}), ("Host 2", "File Server", {"user": 0.8, "root": 0.6}),
    ("Host 2", "Data Server", {"user": 0.8, "root": 0.6}), ("Host 3", "Host 2", {"user": 0.8, "root": 0.8}),
    ("Host 3", "File Server", {"user": 0.8, "root": 0.6}), ("Host 3", "Data Server", {"user": 0.8, "root": 0.6}),
    ("Web Server", "File Server", {"user": 0.8, "root": 0.04}), ("Web Server", "Data Server", {"user": 0.8, "root": 0.04}),
    ("File Server", "Data Server", {"user": 0.8, "root": 0.04}), ("Data Server", "File Server", {"user": 0.6, "root": 0.02}),
    ("Router", "Web Server", {"user": 0.9, "root": 0.9}), ("Web Server", "Router", {"user": 0.9, "root": 0.9}),
    ("Router", "Data Server", {"user": 0.9, "root": 0.1}), ("Data Server", "Router", {"user": 0.9, "root": 0.9}),
    ("Pad", "Router", {"user": 0.7, "root": 0.5}),
]
G_1.add_edges_from(edges)


g1 = dgl.from_networkx(G_1)
# ----- QUAN TRỌNG: Lưu các biến ánh xạ (Tự động cập nhật) -----
node_order = list(G_1.nodes()) # Lưu lại thứ tự node để hiển thị
node_map = {name: i for i, name in enumerate(node_order)} # Tạo map từ Tên -> Index
original_edges_list = list(G_1.edges()) # Lưu lại thứ tự cạnh
# ------------------------------------------
nfeats1 = torch.tensor([[G_1.nodes[n]['state'], G_1.nodes[n]['priority']] for n in node_order], dtype=torch.float32)
efeats1 = torch.tensor([[d['user'], d['root']] for u, v, d in G_1.edges(data=True)], dtype=torch.float32)
g1.ndata['h'] = nfeats1
g1.edata['h'] = efeats1

# --- 2. Tạo Đồ thị 2 (Phức tạp hơn) ---
# (Cấu trúc mạng văn phòng nhỏ, giờ có thêm WiFi, DB nội bộ và Printer)
G_2 = nx.DiGraph()
G_2.add_nodes_from([
    "WAN", "DMZ Server", "User PC 1", "User PC 2",
    "WiFi AP", "Internal DB", "User PC 3", "Printer" # <<< CÁC NODE MỚI
])
node_attr_2 = {
    "WAN": {"state": 0, "priority": 1},
    "DMZ Server": {"state": 1, "priority": 1},
    "User PC 1": {"state": 0, "priority": 0},
    "User PC 2": {"state": 0, "priority": 0},
    # <<< THUỘC TÍNH CHO NODE MỚI
    "WiFi AP": {"state": 0, "priority": 0},
    "Internal DB": {"state": 0, "priority": 1}, # DB nội bộ quan trọng
    "User PC 3": {"state": 0, "priority": 2},
    "Printer": {"state": 0, "priority": 2}
}
nx.set_node_attributes(G_2, node_attr_2)

# <<< DANH SÁCH CẠNH ĐÃ CẬP NHẬT (PHỨC TẠP HƠN)
edges_2 = [
    ("WAN", "DMZ Server", {"user": 0.9, "root": 0.8}),

    # <<< CẠNH MỚI: DMZ cấp kết nối cho WiFi và DB
    ("DMZ Server", "WiFi AP", {"user": 0.8, "root": 0.5}),
    ("DMZ Server", "Internal DB", {"user": 0.7, "root": 0.6}),

    # <<< CẠNH MỚI: WiFi AP cấp kết nối cho các máy con
    ("WiFi AP", "User PC 1", {"user": 0.9, "root": 0.1}),
    ("WiFi AP", "User PC 2", {"user": 0.9, "root": 0.1}),
    ("WiFi AP", "User PC 3", {"user": 0.9, "root": 0.1}),

    # <<< CẠNH MỚI: Tương tác mạng nội bộ
    ("User PC 1", "User PC 2", {"user": 0.3, "root": 0.6}),
    ("User PC 1", "Internal DB", {"user": 0.5, "root": 0.3}), # User truy cập DB
    ("User PC 2", "Internal DB", {"user": 0.5, "root": 0.3}),
    ("User PC 3", "Internal DB", {"user": 0.5, "root": 0.3}),
    ("User PC 1", "Printer", {"user": 0.8, "root": 0.1}), # User in ấn
    ("User PC 2", "Printer", {"user": 0.8, "root": 0.1}),
    ("User PC 3", "Printer", {"user": 0.8, "root": 0.1}),
]
G_2.add_edges_from(edges_2)
g2 = dgl.from_networkx(G_2)
nfeats2_nodes = list(G_2.nodes())
nfeats2 = torch.tensor([[G_2.nodes[n]['state'], G_2.nodes[n]['priority']] for n in nfeats2_nodes], dtype=torch.float32)
efeats2 = torch.tensor([[d['user'], d['root']] for u, v, d in G_2.edges(data=True)], dtype=torch.float32)
g2.ndata['h'] = nfeats2
g2.edata['h'] = efeats2

# --- 3. In kết quả ---
print("--- ĐỒ THỊ 1 (MẠNG CÔNG TY) ---")
print(f"Đã tạo G1: {g1.num_nodes()} nodes, {g1.num_edges()} edges. Đặc trưng node: {nfeats1.shape}")
print(f"\nNode Order G1 (Ánh xạ Index -> Tên):\n{node_order}")
print(f"\nNode Map G1 (Ánh xạ Tên -> Index):\n{node_map}")

print("\n" + "="*30 + "\n")

print("--- ĐỒ THỊ 2 (MẠNG VĂN PHÒNG) ---")
print(f"Đã tạo G2: {g2.num_nodes()} nodes, {g2.num_edges()} edges. Đặc trưng node: {nfeats2.shape}")
print(f"\nNode Order G2:\n{nfeats2_nodes}")

--- ĐỒ THỊ 1 (MẠNG CÔNG TY) ---
Đã tạo G1: 8 nodes, 23 edges. Đặc trưng node: torch.Size([8, 2])

Node Order G1 (Ánh xạ Index -> Tên):
['Data Server', 'Pad', 'Web Server', 'Host 1', 'Host 2', 'Host 3', 'File Server', 'Router']

Node Map G1 (Ánh xạ Tên -> Index):
{'Data Server': 0, 'Pad': 1, 'Web Server': 2, 'Host 1': 3, 'Host 2': 4, 'Host 3': 5, 'File Server': 6, 'Router': 7}


--- ĐỒ THỊ 2 (MẠNG VĂN PHÒNG) ---
Đã tạo G2: 8 nodes, 13 edges. Đặc trưng node: torch.Size([8, 2])

Node Order G2:
['WAN', 'DMZ Server', 'User PC 1', 'User PC 2', 'WiFi AP', 'Internal DB', 'User PC 3', 'Printer']


In [4]:

# 1. Định nghĩa các hằng số
MAX_N_FEATURES = 2
MAX_E_FEATURES = 2

def build_batch_tensor(feats, max_dim):    
    return feats[:, :max_dim]

# 4. Tạo Batch huấn luyện (Sử dụng logic mới)
print("--- Bắt đầu chuẩn hóa & đệm batch ---")

# Xử lý Node Features
nfeats_batch = build_batch_tensor(nfeats1, MAX_N_FEATURES)
nfeats_batch = nfeats_batch.to(device)

# Xử lý Edge Features
efeats_batch = build_batch_tensor(efeats1, MAX_E_FEATURES)
efeats_batch = efeats_batch.to(device)



# Batch các đồ thị
g_batch = dgl.batch([g1])
g_batch = g_batch.to(device)

print(f"\nĐã tạo batch huấn luyện: {g_batch.num_nodes()} nodes, {g_batch.num_edges()} edges")
print(f"Batch feature dims (đã đệm): Node={nfeats_batch.shape}, Edge={efeats_batch.shape}")

--- Bắt đầu chuẩn hóa & đệm batch ---

Đã tạo batch huấn luyện: 8 nodes, 23 edges
Batch feature dims (đã đệm): Node=torch.Size([8, 2]), Edge=torch.Size([23, 2])


In [5]:
# --- Training loop (SỬA ĐỔI VỚI PADDING CHUẨN HÓA) ---

# Lấy kích thước đầu vào TỐI ĐA (MAX_FEATURES)
NDIM_IN = MAX_N_FEATURES # 50
EDIM = MAX_E_FEATURES    # 50

# Lấy các tham số từ cell cũ của bạn
N_HIDDEN = 16
N_OUT = 24
N_LAYERS = 2
EPOCHS = 2000 
LEARNING_RATE = 0.001

# --- Thêm tham số ---
NUM_EXPERTS = 4 # Số lượng chuyên gia
TOP_K = 1       # Định tuyến Top-k

# Khởi tạo encoder và mô hình DGI (SỬ DỤNG CÁC LỚP MỚI)
encoder = EGraphSAGE(
    NDIM_IN, EDIM, N_HIDDEN, N_OUT, N_LAYERS, 
    F.leaky_relu
)
dgi_model = DGI(encoder)
dgi_model = dgi_model.to(device)

# Khởi tạo optimizer
optimizer = torch.optim.Adam(dgi_model.parameters(), lr=LEARNING_RATE)

print("\n--- Bắt đầu quá trình huấn luyện DGI ---")
print(f"Kích thước đầu vào mô hình: Node={NDIM_IN}, Edge={EDIM}")

for epoch in range(EPOCHS):
    dgi_model.train()  # Chuyển mô hình sang chế độ huấn luyện
    optimizer.zero_grad()

    loss = dgi_model(g_batch, nfeats_batch, efeats_batch)

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 200 == 0: 
        print(f'Epoch [{epoch+1}/{EPOCHS}], Loss: {loss.item():.4f}')

print("--- Huấn luyện hoàn tất! ---")


--- Bắt đầu quá trình huấn luyện DGI ---
Kích thước đầu vào mô hình: Node=2, Edge=2
Epoch [200/2000], Loss: 0.3438
Epoch [400/2000], Loss: 0.0088
Epoch [600/2000], Loss: 0.1956
Epoch [800/2000], Loss: 0.0078
Epoch [1000/2000], Loss: 0.2280
Epoch [1200/2000], Loss: 0.0056
Epoch [1400/2000], Loss: 0.0057
Epoch [1600/2000], Loss: 0.0008
Epoch [1800/2000], Loss: 0.0002
Epoch [2000/2000], Loss: 0.0008
--- Huấn luyện hoàn tất! ---


In [10]:
import torch
import yaml
import os

# Đảm bảo thư mục tồn tại
os.makedirs("graphs", exist_ok=True)

# ... (Giả sử các biến NDIM_IN, EDIM... đã được định nghĩa ở trên) ...

# Định nghĩa cấu hình Model
model_config = {
    "NDIM_IN": NDIM_IN,
    "EDIM": EDIM,
    "N_HIDDEN": N_HIDDEN,
    "N_OUT": N_OUT,
    "N_LAYERS": N_LAYERS,
}

# --- PHẦN THAY ĐỔI: Lưu Config bằng YAML ---
CONFIG_PATH = "graphs/model_config.yaml" # Đổi đuôi file

with open(CONFIG_PATH, 'w') as f:
    # default_flow_style=False để file yaml xuống dòng đẹp mắt
    yaml.dump(model_config, f, default_flow_style=False)

print(f"Đã lưu Cấu hình Model (YAML) vào: {CONFIG_PATH}")

# --- PHẦN GIỮ NGUYÊN: Lưu Trọng số Model (vẫn dùng torch.save) ---
MODEL_STATE_PATH = "graphs/dgi_model_state_dict.pth"
torch.save(dgi_model.state_dict(), MODEL_STATE_PATH)
print(f"Đã lưu Trọng số Model vào: {MODEL_STATE_PATH}")

Đã lưu Cấu hình Model (YAML) vào: graphs/model_config.yaml
Đã lưu Trọng số Model vào: graphs/dgi_model_state_dict.pth


In [7]:
# # 1. Đặt tên thư mục (giống như trong Original.ipynb)
# experiment_id = 1 # Bạn có thể thay đổi ID này
# BASE_PATH = f'graphs/{experiment_id}'
# os.makedirs(BASE_PATH, exist_ok=True)
# print(f"Sẽ lưu vào thư mục: {BASE_PATH}")
#
# # 2. Xác định các đường dẫn file
# ENV_PATH = f"{BASE_PATH}/graph_environment.pth"
# CONFIG_PATH = f"{BASE_PATH}/model_config.pth" # File cấu hình GNN
#
# # 5. Lưu Môi trường Tĩnh (dạng .pth)
# # Bao gồm các thông tin phi-tensor mà RL agent cần
# env_data = {
#     "G_original": G_1,     # Đồ thị NetworkX
#     "g1": g1,                     # Đồ thị DGL gốc (chưa xử lý)
#     "node_features_original": nfeats1, # Đặc trưng node gốc (tensor [8, 2])
#     "edge_features_original": efeats1, # Đặc trưng cạnh gốc (tensor [25, 2])
#     "node_order": node_order,     # List: ["Data Server", "Pad", ...]
#     "node_map": node_map,       # Dict: {"Data Server": 0, ...}
# }
# torch.save(env_data, ENV_PATH)
# print(f"Đã lưu Môi trường Tĩnh (ánh xạ) vào: {ENV_PATH}")
#
#
# print("\n--- XUẤT DỮ LIỆU HOÀN TẤT! ---")
# print(f"Giờ bạn có thể chạy sổ tay 'Original.ipynb' với experiment_id = {experiment_id}")

In [8]:
# # --- Bắt đầu xuất file cho RL Agent (Đồ thị G_2) ---
# print("\n--- Bắt đầu xuất file cho RL Agent (Đồ thị G_2) ---")
#
# # 1. Đặt tên thư mục MỚI
# experiment_id_g2 = 2 # ID mới cho đồ thị thứ 2
# BASE_PATH_G2 = f'graphs/{experiment_id_g2}'
# os.makedirs(BASE_PATH_G2, exist_ok=True)
# print(f"Sẽ lưu vào thư mục: {BASE_PATH_G2}")
#
# # 2. Xác định các đường dẫn file MỚI
#
# ENV_PATH_G2 = f"{BASE_PATH_G2}/graph_environment.pth"
#
# # 4. Chuẩn bị và Lưu Môi trường Tĩnh của G_2 (dạng .pth)
# # (Lấy các biến gốc của G_2 từ cell 'define_graphs_cell')
# node_order_g2 = nfeats2_nodes # Biến này đã được tạo trong cell define_graphs
# node_map_g2 = {name: i for i, name in enumerate(node_order_g2)}
# original_edges_list_g2 = list(G_2.edges())
#
# env_data_g2 = {
#     "G_original": G_2,     # Đồ thị NetworkX của G_2
#     "g1": g2,                     # Đồ thị DGL của G_2 (đã chuyển sang device)
#     "node_features_original": nfeats2, # Đặc trưng node gốc của G_2
#     "edge_features_original": efeats2, # Đặc trưng cạnh gốc của G_2
#     "node_order": node_order_g2,     # List: ["WAN", "DMZ Server", ...]
#     "node_map": node_map_g2,       # Dict: {"WAN": 0, ...}
# }
# torch.save(env_data_g2, ENV_PATH_G2)
# print(f"Đã lưu Môi trường Tĩnh (G2) vào: {ENV_PATH_G2}")
#
# print("\n--- XUẤT DỮ LIỆU G_2 HOÀN TẤT! ---")
# print(f"Giờ bạn có thể chạy sổ tay 'Original.ipynb' với experiment_id = {experiment_id_g2}")

In [9]:
# # --- 1. Tạo dữ liệu cho hai đồ thị mạng khác nhau ---
# # (Đây là code GỐC từ yêu cầu của bạn)
#
# # Đồ thị 1 (Lấy từ Graph.ipynb của bạn)
# G_1 = nx.DiGraph()
# G_1.add_nodes_from(["Data Server", "Pad", "Web Server", "Host 1", "Host 2", "Host 3", "File Server", "Router"])
# node_attributes = {
#     "Data Server": {"state": 0, "priority": 2}, "Pad": {"state": 0, "priority": 1},
#     "Web Server": {"state": 0, "priority": 1}, "Host 1": {"state": 0, "priority": 1},
#     "Host 2": {"state": 0, "priority": 0}, "Host 3": {"state": 0, "priority": 0},
#     "File Server": {"state": 0, "priority": 0}, "Router": {"state": 0, "priority": 2}
# }
# nx.set_node_attributes(G_1, node_attributes)
# edges = [
#     ("Pad", "Host 1", {"user": 0.6, "root": 0.48}), ("Pad", "Host 2", {"user": 0.32, "root": 0.32}),
#     ("Pad", "Host 3", {"user": 0.32, "root": 0.32}), ("Pad", "Web Server", {"user": 0.8, "root": 0.6}),
#     ("Host 1", "Pad", {"user": 0.6, "root": 0.6}), ("Host 1", "Web Server", {"user": 0.8, "root": 0.6}),
#     ("Host 1", "Host 2", {"user": 0.32, "root": 0.32}), ("Host 1", "Host 3", {"user": 0.32, "root": 0.32}),
#     ("Host 2", "Host 3", {"user": 0.8, "root": 0.8}), ("Host 2", "File Server", {"user": 0.8, "root": 0.6}),
#     ("Host 2", "Data Server", {"user": 0.8, "root": 0.6}), ("Host 3", "Host 2", {"user": 0.8, "root": 0.8}),
#     ("Host 3", "File Server", {"user": 0.8, "root": 0.6}), ("Host 3", "Data Server", {"user": 0.8, "root": 0.6}),
#     ("Web Server", "File Server", {"user": 0.8, "root": 0.04}), ("Web Server", "Data Server", {"user": 0.8, "root": 0.04}),
#     ("File Server", "Data Server", {"user": 0.8, "root": 0.04}), ("Data Server", "File Server", {"user": 0.6, "root": 0.02}),
#     ("Router", "Web Server", {"user": 0.9, "root": 0.9}), ("Web Server", "Router", {"user": 0.9, "root": 0.9}),
#     ("Router", "Data Server", {"user": 0.9, "root": 0.1}), ("Data Server", "Router", {"user": 0.9, "root": 0.9}),
#     ("Pad", "Router", {"user": 0.7, "root": 0.5}),
# ]
# G_1.add_edges_from(edges)
#
#
# g1 = dgl.from_networkx(G_1)
# # ----- QUAN TRỌNG: Lưu các biến ánh xạ -----
# node_order = list(G_1.nodes()) # Lưu lại thứ tự node để hiển thị
# node_map = {name: i for i, name in enumerate(node_order)} # Tạo map từ Tên -> Index
# original_edges_list = list(G_1.edges()) # Lưu lại thứ tự cạnh
# # ------------------------------------------
# nfeats1 = torch.tensor([[G_1.nodes[n]['state'], G_1.nodes[n]['priority']] for n in node_order], dtype=torch.float32)
# efeats1 = torch.tensor([[d['user'], d['root']] for u, v, d in G_1.edges(data=True)], dtype=torch.float32)
# g1.ndata['h'] = nfeats1
# g1.edata['h'] = efeats1
#
# # 3. Tạo Đồ thị 2 (ví dụ: mạng văn phòng nhỏ với thang đo đặc trưng khác)
# G_2 = nx.DiGraph()
# G_2.add_nodes_from(["WAN", "DMZ Server", "User PC 1", "User PC 2"])
# node_attr_2 = {
#     "WAN": {"state": 0, "priority": 2},
#     "DMZ Server": {"state": 1, "priority": 1},
#     "User PC 1": {"state": 0, "priority": 0},
#     "User PC 2": {"state": 0, "priority": 0}
# }
# nx.set_node_attributes(G_2, node_attr_2)
# edges_2 = [
#     ("WAN", "DMZ Server", {"user": 0.9, "root": 0.8}),
#     ("DMZ Server", "User PC 1", {"user": 0.6, "root": 0.5}),
#     ("DMZ Server", "User PC 2", {"user": 0.2, "root": 0.5}),
#     ("User PC 1", "User PC 2", {"user": 0.3, "root": 0.6})
# ]
# G_2.add_edges_from(edges_2)
# g2 = dgl.from_networkx(G_2)
# nfeats2 = torch.tensor([[G_2.nodes[n]['state'], G_2.nodes[n]['priority']] for n in G_2.nodes()], dtype=torch.float32)
# efeats2 = torch.tensor([[d['user'], d['root']] for u, v, d in G_2.edges(data=True)], dtype=torch.float32)
# g2.ndata['h'] = nfeats2
# g2.edata['h'] = efeats2
#
# print(f"Đã tạo G1: {g1.num_nodes()} nodes, {g1.num_edges()} edges. Đặc trưng: {nfeats1.shape}")
# print(f"Đã tạo G2: {g2.num_nodes()} nodes, {g2.num_edges()} edges. Đặc trưng: {nfeats2.shape}")
# print(f"\nNode Order (Ánh xạ Index -> Tên):\n{node_order}")
# print(f"\nNode Map (Ánh xạ Tên -> Index):\n{node_map}")

**LƯU Ý VỀ CÁC CELL KHÁC:**

Các cell còn lại trong sổ tay của bạn (ví dụ: các cell trực quan hóa node và heatmap) có thể được giữ nguyên. Chúng đều hoạt động dựa trên biến `final_node_embeddings` và `final_edge_embeddings` được tạo ra trong cell `inference_cell`.